In [1]:
from time import strftime,localtime
import pandas as pd
from os.path import exists

In [2]:
df = pd.read_html("https://www.espn.com/nba/stats/player/_/season/2021/seasontype/2/table/offensive/sort/avgPoints/dir/desc")

In [3]:
dfs = df[0].join(df[1])

In [4]:
dfs[['Name','Team']] = dfs['Name'].str.extract('^(.*?)([A-Z]+)$', expand=True)

In [5]:
dfs.head()

,RK,Name,POS,GP,MIN,PTS,FGM,FGA,FG%,3PM,...,FT%,REB,AST,STL,BLK,TO,DD2,TD3,PER,Team
0,1,Stephen Curry,PG,63,34.2,32.0,10.4,21.7,48.2,5.3,...,91.6,5.5,5.8,1.2,0.1,3.4,8,0,26.37,GS
1,2,Bradley Beal,SG,60,35.8,31.3,11.2,23.0,48.5,2.2,...,88.9,4.7,4.4,1.2,0.4,3.1,4,0,22.81,WSH
2,3,Damian Lillard,PG,67,35.8,28.8,9.0,19.9,45.1,4.1,...,92.8,4.2,7.5,0.9,0.3,3.0,16,0,25.65,POR
3,4,Joel Embiid,C,51,31.1,28.5,9.0,17.6,51.3,1.1,...,85.9,10.6,2.8,1.0,1.4,3.1,31,0,30.32,PHI
4,5,Giannis Antetokounmpo,PF,61,33.0,28.1,10.3,18.0,56.9,1.1,...,68.5,11.0,5.9,1.2,1.2,3.4,41,7,29.24,MIL


In [6]:
def get_team_stats(teams, last_n_years, include_postseason = False):
    this_year = int(strftime("%Y",localtime()))

In [19]:
class Team_standings:
    
    def _init_(self):
        pass
    
    
    def build_table(self,url):
        dfs = pd.read_html(url)
        stats = dfs[1] 
        teams = dfs[0]
        teams = teams.rename(columns={0:"Team"})

        return teams.join(stats)
    
    def gen_key(self, year, season_type):
        return year+"_"+season_type

    def build_url(self, root,year_n, tail,preseason = False):
        if(not preseason):
            return root +  "/" + year_n + tail
        else:
            return root + "/" + year_n + tail
        
    def update_team_standings(self):
        REG = 0
        PRE = 1
        labels =("regular_season", "pre_season")
        this_year = strftime("%Y",localtime())

        
        root = "https://www.espn.com/nba/standings/_"
        tail = "/group/league"
        
        filename = self.gen_key(this_year, labels[REG]) + ".csv"
        filepath = "./team_standings/reg_season/" +filename

        this_year_reg_season = root + tail
        table = self.build_table(this_year_reg_season)
        table.to_csv(filepath, index = False)
        
        filename = self.gen_key(this_year, labels[Pre]) + ".csv"
        filepath = "./team_standings/pre_season/" +filename
        
        this_year_pre_season = root + "/seasontype/pre" + tail
        table = self.build_table(this_year_pre_season)
        table.to_csv(filepath, index = False)
        
        return
    
    def get_team_standings(self, last_n_years, include_preseason = False):
        REG = 0
        PRE = 1
        labels =("regular_season", "pre_season")
        this_year = strftime("%Y",localtime())

        root = "https://www.espn.com/nba/standings/_"
        tail = "/group/league"

        
        year = int(this_year)
        reg_season_root = root + "/season"
        preseason_root = root + "/seasontype/pre/season"

        for y in range(year,year-last_n_years,-1):
            filename = self.gen_key(str(y-1),labels[REG]) + ".csv"
            filepath = "./team_standings/reg_season/" + filename
            
            if(not exists(filepath)):
#                 print("New file")
                reg_season = self.build_url(reg_season_root, str(y-1), tail)
                table = self.build_table(reg_season)

                table.to_csv(filepath, index = False)

            filename = self.gen_key(str(y-1),labels[PRE]) + ".csv"
            filepath = "./team_standings/pre_season/" + filename

            if(not exists(filepath)):
#                 print("New file")
                pre_season = self.build_url(preseason_root, str(y-1), tail,True)
                table = self.build_table(pre_season)

                table.to_csv(filepath,index = False)
                
        return 

In [20]:
standings_scraper = Team_standings()

In [21]:
standings_scraper.get_team_standings(5, True)

In [22]:
standings_scraper.update_team_standings()

NameError: name 'Pre' is not defined